In [3]:
import pandas as pd
import numpy as np
np.random.seed(0)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# EDA part 2:
- josh's confusion matrix predicted everything along Assault. all our falses were basically in Assault
    - Assault was 4200/5000, so this is to be expected w/ large class imbalance
    - 2 solutions:
        - more data
        - smote
- let's also look at doing SVC to pick out our variables
    

## SVM attempts

In [ ]:
print('importing')
df = pd.read_csv('merged.csv')
df = df.drop(['Unnamed: 0','state_y','state_x'], axis=1) # drop unneeded cols
# df = df.drop(to_drop,axis=1)

In [ ]:
print('begin sample')
# sample down to 30k
df = df.sample(30000, random_state=1)
print('sample complete')
# drop nulls?
print('dropping nulls')
df = df.dropna()

In [ ]:





# df with dummies
print('getting dummies')
# remove Y to avoid Y dummies
target = df['UCR_code']
df = df.drop('UCR_code',axis=1)

# dummify
df = pd.get_dummies(df,drop_first=True)

features = list(df.columns) # get dummy list

df['UCR_code'] = target # this should replace the original undummied Y
print('done with imports & cleaning')

In [1]:
from time import time

In [ ]:
X = bankdata.drop('Class', axis=1)  
y = bankdata['Class'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) 

In [ ]:
from sklearn.svm import SVC  
tic = time()
svclassifier = SVC(kernel='rbf', C=0.05)  
svclassifier.fit(X_train, y_train) 
y_pred = svclassifier.predict(X_test)
toc = time()
print("run time is {} seconds".format(toc-tic))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred)) 
print("The accuracy score is" + " "+ str(accuracy_score(y_test, y_pred)))